In [1]:
## This file implements neural networks and random forest on p11presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 90.85% for over-sampling data, 92.07% after improvement.
## And it is 97.17% for combination data and 95.28% with dropout.
## For random forest, the accuracy is 90.24% for over-sampling data and 99.06% for combination data.
## For random forest with cross-validation, the mean accuracy is 85.91% for over-sampling, and 96.75% for combination data.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p11presabs_qual.csv')
df.shape

(255, 974)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      1
2      2
3      2
4      2
5      2
6      2
7      2
8      2
9      2
10     1
11     2
12     2
13     1
14     2
15     2
16     0
17     0
18     2
19     0
20     0
21     0
22     1
23     2
24     2
25     0
26     2
27     0
28     2
29     2
      ..
225    2
226    1
227    2
228    2
229    2
230    1
231    2
232    2
233    2
234    2
235    2
236    2
237    2
238    2
239    2
240    2
241    2
242    0
243    2
244    2
245    2
246    2
247    2
248    2
249    2
250    2
251    1
252    2
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

2    182
1     48
0     25
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 973)

In [9]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 972) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 182), (1, 182), (2, 182)]


Using TensorFlow backend.


In [12]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 164), (1, 111), (2, 77)]


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [17]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/100
382/382 [==============================] - 0s 323us/step - loss: 1.2176 - accuracy: 0.4188 - val_loss: 0.9369 - val_accuracy: 0.5976
Epoch 2/100
382/382 [==============================] - 0s 94us/step - loss: 0.8125 - accuracy: 0.6545 - val_loss: 0.7341 - val_accuracy: 0.6402
Epoch 3/100
382/382 [==============================] - 0s 99us/step - loss: 0.7441 - accuracy: 0.6675 - val_loss: 0.6780 - val_accuracy: 0.6890
Epoch 4/100
382/382 [==============================] - 0s 95us/step - loss: 0.6756 - accuracy: 0.6937 - val_loss: 0.7092 - val_accuracy: 0.6707
Epoch 5/100
382/382 [==============================] - 0s 100us/step - loss: 0.6181 - accuracy: 0.7330 - val_loss: 0.6035 - val_accuracy: 0.7622
Epoch 6/100
382/382 [==============================] - 0s 89us/step - loss: 0.5997 - accuracy: 0.7539 - val_loss: 0.5969 - val_accuracy: 0.7866
Epoch 7/100
382/382 [==============================] - 0s 94us/step - loss: 0.5780 - acc

382/382 [==============================] - 0s 135us/step - loss: 0.1909 - accuracy: 0.9241 - val_loss: 0.3367 - val_accuracy: 0.8537
Epoch 58/100
382/382 [==============================] - 0s 141us/step - loss: 0.1766 - accuracy: 0.9398 - val_loss: 0.3340 - val_accuracy: 0.8537
Epoch 59/100
382/382 [==============================] - 0s 109us/step - loss: 0.1959 - accuracy: 0.9215 - val_loss: 0.3142 - val_accuracy: 0.8659
Epoch 60/100
382/382 [==============================] - 0s 100us/step - loss: 0.1750 - accuracy: 0.9293 - val_loss: 0.3172 - val_accuracy: 0.9024
Epoch 61/100
382/382 [==============================] - 0s 110us/step - loss: 0.1552 - accuracy: 0.9450 - val_loss: 0.3037 - val_accuracy: 0.9024
Epoch 62/100
382/382 [==============================] - 0s 95us/step - loss: 0.1552 - accuracy: 0.9450 - val_loss: 0.3238 - val_accuracy: 0.8841
Epoch 63/100
382/382 [==============================] - 0s 97us/step - loss: 0.1427 - accuracy: 0.9634 - val_loss: 0.3379 - val_accuracy: 

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

164/164 [==============================] - 0s 79us/step
over-sampling test accuracy: 90.85%


In [25]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [26]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/1000
382/382 [==============================] - 0s 494us/step - loss: 3.3873 - accuracy: 0.4529 - val_loss: 0.9816 - val_accuracy: 0.5061
Epoch 2/1000
382/382 [==============================] - 0s 117us/step - loss: 3.3668 - accuracy: 0.4555 - val_loss: 0.8844 - val_accuracy: 0.5427
Epoch 3/1000
382/382 [==============================] - 0s 101us/step - loss: 3.0283 - accuracy: 0.5471 - val_loss: 0.8332 - val_accuracy: 0.5854
Epoch 4/1000
382/382 [==============================] - 0s 103us/step - loss: 4.1540 - accuracy: 0.4791 - val_loss: 0.8036 - val_accuracy: 0.6341
Epoch 5/1000
382/382 [==============================] - 0s 103us/step - loss: 3.9544 - accuracy: 0.5131 - val_loss: 0.8092 - val_accuracy: 0.6524
Epoch 6/1000
382/382 [==============================] - 0s 102us/step - loss: 3.2149 - accuracy: 0.6073 - val_loss: 0.7721 - val_accuracy: 0.6524
Epoch 7/1000
382/382 [==============================] - 0s 99us/step - loss: 2

382/382 [==============================] - 0s 112us/step - loss: 1.0484 - accuracy: 0.7513 - val_loss: 1.3177 - val_accuracy: 0.8293
Epoch 57/1000
382/382 [==============================] - 0s 105us/step - loss: 1.5656 - accuracy: 0.6859 - val_loss: 0.9017 - val_accuracy: 0.8293
Epoch 58/1000
382/382 [==============================] - 0s 95us/step - loss: 1.5784 - accuracy: 0.7120 - val_loss: 0.8871 - val_accuracy: 0.8293
Epoch 59/1000
382/382 [==============================] - 0s 86us/step - loss: 1.5997 - accuracy: 0.7173 - val_loss: 1.2991 - val_accuracy: 0.8232
Epoch 60/1000
382/382 [==============================] - 0s 82us/step - loss: 1.8011 - accuracy: 0.7068 - val_loss: 1.0224 - val_accuracy: 0.7927
Epoch 61/1000
382/382 [==============================] - 0s 82us/step - loss: 1.4076 - accuracy: 0.7618 - val_loss: 1.1802 - val_accuracy: 0.8293
Epoch 62/1000
382/382 [==============================] - 0s 82us/step - loss: 1.2087 - accuracy: 0.7461 - val_loss: 1.1537 - val_accurac

382/382 [==============================] - 0s 83us/step - loss: 0.6472 - accuracy: 0.7880 - val_loss: 1.0469 - val_accuracy: 0.8537
Epoch 113/1000
382/382 [==============================] - 0s 88us/step - loss: 0.7148 - accuracy: 0.7906 - val_loss: 0.8738 - val_accuracy: 0.9085
Epoch 114/1000
382/382 [==============================] - 0s 84us/step - loss: 1.0002 - accuracy: 0.7539 - val_loss: 1.1740 - val_accuracy: 0.8415
Epoch 115/1000
382/382 [==============================] - 0s 83us/step - loss: 0.7711 - accuracy: 0.7592 - val_loss: 1.2492 - val_accuracy: 0.8354
Epoch 116/1000
382/382 [==============================] - 0s 81us/step - loss: 0.9704 - accuracy: 0.7304 - val_loss: 1.2262 - val_accuracy: 0.8293
Epoch 117/1000
382/382 [==============================] - 0s 81us/step - loss: 1.0016 - accuracy: 0.7408 - val_loss: 1.0493 - val_accuracy: 0.8354
Epoch 118/1000
382/382 [==============================] - 0s 84us/step - loss: 1.3061 - accuracy: 0.7513 - val_loss: 1.7300 - val_acc

Epoch 168/1000
382/382 [==============================] - 0s 85us/step - loss: 0.6211 - accuracy: 0.7958 - val_loss: 0.7934 - val_accuracy: 0.9207
Epoch 169/1000
382/382 [==============================] - 0s 90us/step - loss: 0.4172 - accuracy: 0.8298 - val_loss: 1.1144 - val_accuracy: 0.9024
Epoch 170/1000
382/382 [==============================] - 0s 83us/step - loss: 0.5767 - accuracy: 0.8168 - val_loss: 1.4677 - val_accuracy: 0.8598
Epoch 171/1000
382/382 [==============================] - 0s 85us/step - loss: 0.5885 - accuracy: 0.8115 - val_loss: 0.7142 - val_accuracy: 0.9207
Epoch 172/1000
382/382 [==============================] - 0s 83us/step - loss: 0.4069 - accuracy: 0.8298 - val_loss: 0.6816 - val_accuracy: 0.9268
Epoch 173/1000
382/382 [==============================] - 0s 83us/step - loss: 0.5590 - accuracy: 0.7801 - val_loss: 1.2619 - val_accuracy: 0.8598
Epoch 174/1000
382/382 [==============================] - 0s 81us/step - loss: 0.6522 - accuracy: 0.7775 - val_loss: 0

Epoch 224/1000
382/382 [==============================] - 0s 87us/step - loss: 0.6754 - accuracy: 0.7723 - val_loss: 0.6728 - val_accuracy: 0.9329
Epoch 225/1000
382/382 [==============================] - 0s 84us/step - loss: 0.4400 - accuracy: 0.7880 - val_loss: 1.0317 - val_accuracy: 0.9207
Epoch 226/1000
382/382 [==============================] - 0s 82us/step - loss: 0.4252 - accuracy: 0.7853 - val_loss: 1.0626 - val_accuracy: 0.9207
Epoch 227/1000
382/382 [==============================] - 0s 83us/step - loss: 0.5284 - accuracy: 0.7618 - val_loss: 0.9346 - val_accuracy: 0.8902
Epoch 228/1000
382/382 [==============================] - 0s 90us/step - loss: 0.4552 - accuracy: 0.7670 - val_loss: 1.0537 - val_accuracy: 0.8963
Epoch 229/1000
382/382 [==============================] - 0s 87us/step - loss: 0.4466 - accuracy: 0.7984 - val_loss: 0.7366 - val_accuracy: 0.9268
Epoch 230/1000
382/382 [==============================] - 0s 82us/step - loss: 0.5193 - accuracy: 0.7670 - val_loss: 0

...

Epoch 726/1000
382/382 [==============================] - 0s 88us/step - loss: 0.3562 - accuracy: 0.7932 - val_loss: 1.1047 - val_accuracy: 0.9268
Epoch 727/1000
382/382 [==============================] - 0s 86us/step - loss: 0.3588 - accuracy: 0.8063 - val_loss: 1.0305 - val_accuracy: 0.9207
Epoch 728/1000
382/382 [==============================] - 0s 83us/step - loss: 0.4067 - accuracy: 0.7801 - val_loss: 1.1445 - val_accuracy: 0.9207
Epoch 729/1000
382/382 [==============================] - 0s 83us/step - loss: 0.3684 - accuracy: 0.8037 - val_loss: 1.1061 - val_accuracy: 0.9268
Epoch 730/1000
382/382 [==============================] - 0s 82us/step - loss: 0.4251 - accuracy: 0.7775 - val_loss: 1.1614 - val_accuracy: 0.9207
Epoch 731/1000
382/382 [==============================] - 0s 83us/step - loss: 0.3730 - accuracy: 0.8037 - val_loss: 1.1757 - val_accuracy: 0.9207
Epoch 732/1000
382/382 [==============================] - 0s 81us/step - loss: 0.3650 - accuracy: 0.7958 - val_loss: 1

Epoch 782/1000
382/382 [==============================] - 0s 87us/step - loss: 0.2774 - accuracy: 0.8272 - val_loss: 1.0116 - val_accuracy: 0.9207
Epoch 783/1000
382/382 [==============================] - 0s 86us/step - loss: 0.3800 - accuracy: 0.8063 - val_loss: 1.0276 - val_accuracy: 0.9268
Epoch 784/1000
382/382 [==============================] - 0s 93us/step - loss: 0.3503 - accuracy: 0.8141 - val_loss: 1.1050 - val_accuracy: 0.9268
Epoch 785/1000
382/382 [==============================] - 0s 82us/step - loss: 0.4386 - accuracy: 0.7277 - val_loss: 1.2937 - val_accuracy: 0.9024
Epoch 786/1000
382/382 [==============================] - 0s 84us/step - loss: 0.3624 - accuracy: 0.7984 - val_loss: 1.0772 - val_accuracy: 0.9268
Epoch 787/1000
382/382 [==============================] - 0s 91us/step - loss: 0.3866 - accuracy: 0.7592 - val_loss: 1.1376 - val_accuracy: 0.9268
Epoch 788/1000
382/382 [==============================] - 0s 82us/step - loss: 0.5508 - accuracy: 0.7382 - val_loss: 1

Epoch 838/1000
382/382 [==============================] - 0s 94us/step - loss: 0.3540 - accuracy: 0.7958 - val_loss: 1.0449 - val_accuracy: 0.9268
Epoch 839/1000
382/382 [==============================] - 0s 93us/step - loss: 0.4213 - accuracy: 0.7958 - val_loss: 1.0761 - val_accuracy: 0.9268
Epoch 840/1000
382/382 [==============================] - 0s 86us/step - loss: 0.3492 - accuracy: 0.8063 - val_loss: 1.0564 - val_accuracy: 0.9268
Epoch 841/1000
382/382 [==============================] - 0s 97us/step - loss: 0.3626 - accuracy: 0.7749 - val_loss: 1.0596 - val_accuracy: 0.9268
Epoch 842/1000
382/382 [==============================] - 0s 86us/step - loss: 0.3198 - accuracy: 0.7958 - val_loss: 1.1102 - val_accuracy: 0.9268
Epoch 843/1000
382/382 [==============================] - 0s 82us/step - loss: 0.4315 - accuracy: 0.7644 - val_loss: 1.1666 - val_accuracy: 0.9268
Epoch 844/1000
382/382 [==============================] - 0s 82us/step - loss: 0.4116 - accuracy: 0.8037 - val_loss: 1

Epoch 894/1000
382/382 [==============================] - 0s 86us/step - loss: 0.3049 - accuracy: 0.8089 - val_loss: 1.1103 - val_accuracy: 0.9268
Epoch 895/1000
382/382 [==============================] - 0s 84us/step - loss: 0.4278 - accuracy: 0.7853 - val_loss: 1.1142 - val_accuracy: 0.9085
Epoch 896/1000
382/382 [==============================] - 0s 82us/step - loss: 0.4342 - accuracy: 0.8298 - val_loss: 1.2646 - val_accuracy: 0.9085
Epoch 897/1000
382/382 [==============================] - 0s 81us/step - loss: 0.3914 - accuracy: 0.8063 - val_loss: 1.1464 - val_accuracy: 0.9268
Epoch 898/1000
382/382 [==============================] - 0s 85us/step - loss: 0.3984 - accuracy: 0.7906 - val_loss: 1.9828 - val_accuracy: 0.8415
Epoch 899/1000
382/382 [==============================] - 0s 91us/step - loss: 0.8381 - accuracy: 0.7461 - val_loss: 1.6165 - val_accuracy: 0.8720
Epoch 900/1000
382/382 [==============================] - 0s 98us/step - loss: 0.3879 - accuracy: 0.8272 - val_loss: 1

Epoch 950/1000
382/382 [==============================] - 0s 89us/step - loss: 0.4064 - accuracy: 0.8010 - val_loss: 1.0888 - val_accuracy: 0.9207
Epoch 951/1000
382/382 [==============================] - 0s 85us/step - loss: 0.3754 - accuracy: 0.7958 - val_loss: 1.1421 - val_accuracy: 0.9268
Epoch 952/1000
382/382 [==============================] - 0s 83us/step - loss: 0.3483 - accuracy: 0.7775 - val_loss: 1.0466 - val_accuracy: 0.9085
Epoch 953/1000
382/382 [==============================] - 0s 82us/step - loss: 0.3609 - accuracy: 0.7723 - val_loss: 1.1428 - val_accuracy: 0.9268
Epoch 954/1000
382/382 [==============================] - 0s 82us/step - loss: 0.3694 - accuracy: 0.8089 - val_loss: 1.0485 - val_accuracy: 0.9085
Epoch 955/1000
382/382 [==============================] - 0s 90us/step - loss: 0.3916 - accuracy: 0.7801 - val_loss: 1.5657 - val_accuracy: 0.8841
Epoch 956/1000
382/382 [==============================] - 0s 83us/step - loss: 0.3377 - accuracy: 0.8560 - val_loss: 1

In [28]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

164/164 [==============================] - 0s 85us/step
over-sampling test accuracy: 92.07%


In [29]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [30]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 246 samples, validate on 106 samples
Epoch 1/100
246/246 [==============================] - 0s 449us/step - loss: 0.8128 - accuracy: 0.7358 - val_loss: 0.7226 - val_accuracy: 0.7830
Epoch 2/100
246/246 [==============================] - 0s 109us/step - loss: 0.6270 - accuracy: 0.7927 - val_loss: 0.4628 - val_accuracy: 0.8491
Epoch 3/100
246/246 [==============================] - 0s 122us/step - loss: 0.4346 - accuracy: 0.8699 - val_loss: 0.3980 - val_accuracy: 0.8491
Epoch 4/100
246/246 [==============================] - 0s 113us/step - loss: 0.3392 - accuracy: 0.8740 - val_loss: 0.4111 - val_accuracy: 0.7925
Epoch 5/100
246/246 [==============================] - 0s 114us/step - loss: 0.3209 - accuracy: 0.8699 - val_loss: 0.3326 - val_accuracy: 0.8302
Epoch 6/100
246/246 [==============================] - 0s 123us/step - loss: 0.2617 - accuracy: 0.8862 - val_loss: 0.2996 - val_accuracy: 0.8679
Epoch 7/100
246/246 [==============================] - 0s 140us/step - loss: 0.2280 

Epoch 57/100
246/246 [==============================] - 0s 194us/step - loss: 0.0250 - accuracy: 0.9919 - val_loss: 0.1213 - val_accuracy: 0.9623
Epoch 58/100
246/246 [==============================] - 0s 129us/step - loss: 0.0249 - accuracy: 1.0000 - val_loss: 0.1067 - val_accuracy: 0.9623
Epoch 59/100
246/246 [==============================] - 0s 112us/step - loss: 0.0279 - accuracy: 1.0000 - val_loss: 0.1308 - val_accuracy: 0.9623
Epoch 60/100
246/246 [==============================] - 0s 116us/step - loss: 0.0259 - accuracy: 0.9959 - val_loss: 0.1327 - val_accuracy: 0.9623
Epoch 61/100
246/246 [==============================] - 0s 98us/step - loss: 0.0290 - accuracy: 1.0000 - val_loss: 0.0945 - val_accuracy: 0.9811
Epoch 62/100
246/246 [==============================] - 0s 100us/step - loss: 0.0253 - accuracy: 1.0000 - val_loss: 0.0972 - val_accuracy: 0.9717
Epoch 63/100
246/246 [==============================] - 0s 90us/step - loss: 0.0213 - accuracy: 1.0000 - val_loss: 0.1183 - v

In [32]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

106/106 [==============================] - 0s 177us/step
combination test accuracy: 97.17%


In [41]:
#### add dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [42]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 246 samples, validate on 106 samples
Epoch 1/100
246/246 [==============================] - 0s 643us/step - loss: 3.3059 - accuracy: 0.5528 - val_loss: 0.7112 - val_accuracy: 0.7170
Epoch 2/100
246/246 [==============================] - 0s 122us/step - loss: 3.3419 - accuracy: 0.6585 - val_loss: 0.6842 - val_accuracy: 0.7170
Epoch 3/100
246/246 [==============================] - 0s 133us/step - loss: 2.9540 - accuracy: 0.6951 - val_loss: 0.4203 - val_accuracy: 0.8208
Epoch 4/100
246/246 [==============================] - 0s 127us/step - loss: 2.7524 - accuracy: 0.6870 - val_loss: 0.4049 - val_accuracy: 0.8019
Epoch 5/100
246/246 [==============================] - 0s 129us/step - loss: 3.0223 - accuracy: 0.7033 - val_loss: 0.5934 - val_accuracy: 0.8019
Epoch 6/100
246/246 [==============================] - 0s 132us/step - loss: 2.3209 - accuracy: 0.7358 - val_loss: 0.4865 - val_accuracy: 0.8019
Epoch 7/100
246/246 [==============================] - 0s 128us/step - loss: 2.5704 

Epoch 57/100
246/246 [==============================] - 0s 129us/step - loss: 0.7865 - accuracy: 0.7967 - val_loss: 0.2680 - val_accuracy: 0.9623
Epoch 58/100
246/246 [==============================] - 0s 114us/step - loss: 0.6011 - accuracy: 0.8008 - val_loss: 0.4664 - val_accuracy: 0.9245
Epoch 59/100
246/246 [==============================] - 0s 113us/step - loss: 0.5261 - accuracy: 0.8089 - val_loss: 0.4148 - val_accuracy: 0.9434
Epoch 60/100
246/246 [==============================] - 0s 110us/step - loss: 0.6433 - accuracy: 0.7967 - val_loss: 0.3213 - val_accuracy: 0.9434
Epoch 61/100
246/246 [==============================] - 0s 99us/step - loss: 0.7643 - accuracy: 0.7967 - val_loss: 0.4826 - val_accuracy: 0.9057
Epoch 62/100
246/246 [==============================] - 0s 100us/step - loss: 0.5939 - accuracy: 0.7642 - val_loss: 0.2521 - val_accuracy: 0.9623
Epoch 63/100
246/246 [==============================] - 0s 99us/step - loss: 0.7676 - accuracy: 0.8130 - val_loss: 0.2909 - v

In [44]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

106/106 [==============================] - 0s 110us/step
combination test accuracy: 95.28%


In [45]:
############## Random Forest ##############

In [46]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 90.24%


In [48]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.06%


In [50]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.76923077 0.88461538 0.88157895 0.82666667 0.93333333]
0.8590850202429149


In [51]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.96       0.96       0.97959184 0.97959184 0.95833333]
0.9675034013605442
